# SNP Cross Disease Analysis

Perform analysis on SNPs linked with specific diseases and produce results and plots. Make sure there is a subfolder named "plots" and subfolders within "plots" named "boxplots", "violinplots", and "scatterplots" to save plots to. (Alternatively, rename where the plots are saved within the code.)

# Disease Analysis

Specific libraries are imported.

In [2]:
import csv
import numpy as np
import collections as col
import random
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import ks_2samp

The lists of diseases and information relevant to files containing SNP effect size for these diseases such as file name, indices in files that contain SNP ID and effect size, and delimiters for the file type are initialized. The 11 specific diseases/traits/disorders analyzed are Alzheimer's, Parkinson's, Schizophrenia, Autism, Depression, Type 2 Diabetes, Body Mass Index, Obesity, Asthma, Smoking, and LDL Cholesterol Levels. The summary statistics for SNP effect size for each disease files are included and are all obtained from links on http://ldsc.broadinstitute.org/gwashare/. For each disease, the effect size may use Beta, which is centered around 0, or Odds Ratio, which is centered around 1.

In [3]:
# List of diseases
diseases = ["Alzheimer", "Parkinson", "Schizophrenia", "Autism", "Depress", "Type 2 Diabetes", "Body Mass Index", "Obesity", "Asthma", "Smok", "LDL"]

numdiseases = len(diseases)

# List of disease summary statistics file names
diseasefiles = ["alzheimerssumstats.txt", "parkinsonssumstats.txt", "schizophreniasumstats.txt", "autismsumstats.txt", "depressionsumstats3.txt", "type2diabetessumstats.txt", "bmisumstats.txt", "obesitysumstats.txt", "asthmasumstats.txt", "smokingsumstats.tbl", "ldlsumstats.txt"]

# List of corresponding indices for snpids and effect sizes for each disease file
fileindices = [(2, 5), (0, 13), (0, 5), (0, 5), (0, 6), (0, 6), (0, 7), (0, 4), (1, 7), (1, 8), (2, 5)]

# List of corresponding file delimiters
filedelimiters = ["\t", "\t", "\t", "\t", "\t", "\t", "\t", " ", "\t", "\t", "\t"]

Additional data structures to store SNPs linked with each disease, SNP effect sizes for each disease, and whether or not there is a known effect size for a SNP for a specific disease are initialized.

In [4]:
# Dictionary of sets of Disease-related SNPs
snps = col.defaultdict(set)

# Dictionary of dicts from SNP to Beta for each disease
allsnpbetas = col.defaultdict(dict)

# Dictionary of disease to dict that tells whether SNP has known effect size from disease files or not
snpsindict = col.defaultdict(lambda: col.defaultdict(lambda: "no"))

# Dictionary of SNP to information about SNP from associations file
snpsinfo = {}

# List of sets of disease-related SNPs for each disease
diseasesnps = []

# List of sets of disease-related SNP betas for the same disease
diseasesnpbetas = []

# Lists of indicies of diseases summary statistics files that use beta vs odds ratio for effect size
betainds = [0, 4, 6, 7, 9, 10]
oddsinds = [1, 2, 3, 5, 8]

The filtered list of associations from SNAP is loaded and SNPs are read and categorized by disease/trait. The number of SNPs linked with each disease is printed.

In [5]:
# Read filtered SNPs and stores SNPs related to the specific diseases
with open("snapfiltered.associations.new2.tsv") as new:
    for line in csv.reader(new, delimiter="\t"):
        snpid = line[1]
        pheno = line[2]
        for disease in diseases:
            if disease.upper() in pheno.upper():
                snps[disease].add(snpid)
                paper = line[0]
                pvalue = line[4]
                snpsinfo[snpid] = (snpid, pheno, paper, pvalue)
                break

snps = {key: list(values) for key, values in snps.items()} # turns dict of sets into dict of lists
numsnps = [len(snps[disease]) for disease in diseases] # list of number of SNPs linked with each disease

for i in range(numdiseases):
    print "Number of " + diseases[i] + " SNPs:", numsnps[i]

Number of Alzheimer SNPs: 95
Number of Parkinson SNPs: 68
Number of Schizophrenia SNPs: 37
Number of Autism SNPs: 32
Number of Depress SNPs: 38
Number of Type 2 Diabetes SNPs: 19
Number of Body Mass Index SNPs: 60
Number of Obesity SNPs: 31
Number of Asthma SNPs: 48
Number of Smok SNPs: 17
Number of LDL SNPs: 16


All of the files that contain SNP effect size for each disease are read and the effect sizes are stored.

In [6]:
# Function to tell if input string is float or not
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

# Read disease files
for i in range(numdiseases):
    disease = diseases[i]
    file = diseasefiles[i]
    snpindex = fileindices[i][0]
    betaindex = fileindices[i][1]
    delim = filedelimiters[i]
    with open(file) as new:
        for line in csv.reader(new, delimiter=delim):
            snpid = line[snpindex]
            if not is_number(line[betaindex]):
                continue
            beta = float(line[betaindex])
            allsnpbetas[disease][snpid] = beta
            snpsindict[disease][snpid] = "yes"

The cross disease analysis is then performed. For each disease 1's set of linked SNPs, for each disease 2, the average effect size of disease 1 linked SNPs is calculated for disease 2 (which leads to 11 average effect sizes for disease 1 SNPs, one for each disease 2). Then a number of random samples of SNPs (1000 in this case) of the same number of disease 1 SNPs for which there was a known effect size for disease 2 are chosen and their average effect sizes are calculated. Using this permutation test, we can determine whether or not the effect size for disease 1 SNPs with respect to disease 2 is significant or not by calculating a simple p-value of (total samples - # random samples where the effect size for the disease 1 sample > effect size for random sample) / (total samples). The number of disease 1 SNPs with known effect size for disease 2, average effect sizes, p-values, and -log(p-values) are stored in matrices. A p-value using the normal approximation is also calculated.

Additionally, for analyses of disease 1 SNPs with the same disease (when disease 1 = disease 2), scatterplots of the mean of the effect sizes of the sample of disease 1 SNPs are plotted along with the means of the 1000 random samples of SNPs for that disease.

In [7]:
# Create list of cross disease matrices for number of disease1 SNPs with known effect size for disease2, average effect sizes, p-values using perumtation test, log p-values, and normal approx. p-values
crossdiseases = [np.zeros((numdiseases, numdiseases)) for i in range(5)]

totalsamples = 1000

# Information used for later plots
allavgsamplebetas = [] # list of lists that contain the mean effect size for the random samples for each disease 
allavgbetas = [] # list mean effect size for the disease linked SNPs with the same disease for each disease

for i in range(numdiseases):
    disease1 = diseases[i]
    snplist = snps[disease1] # disease1 linked SNPs
    for j in range(numdiseases):
        disease2 = diseases[j]
        snpbetas = []
        dsnplist = []
        for snp in snplist:
            if snpsindict[disease2][snp] == "no": # if no effect size for disease2 for a disease1 linked SNP, skip
                continue
            snpbetas.append(allsnpbetas[disease2][snp]) # add effect size of SNP to list
            dsnplist.append(snp) # add SNP name to list

        if i == j: # if disease linked SNPs analyzed with same disease, get information
            diseasesnps.append(dsnplist)
            diseasesnpbetas.append(snpbetas)
        
        # Number of disease1 linked SNPs that have known effect size for disease2 and their average effect sizes
        numsnpbetas = len(snpbetas)
        avgbeta = np.mean(snpbetas)

        # Create random samples of all known SNPs with effect size for disease2
        allsnps = allsnpbetas[disease2].keys()
        avgsamplebetas = []
        for k in range(totalsamples):
            randsample = random.sample(allsnps, numsnpbetas)
            avgsamplebeta = np.mean([allsnpbetas[disease2][snp] for snp in randsample])
            avgsamplebetas.append(avgsamplebeta)

        # Plot Scatter if disease-linked snps with same disease, abs(beta) if using beta and abs(log(OR)) if using odds ratio
        if i == j:
            allavgsamplebetas.append(avgsamplebetas)
            allavgbetas.append(avgbeta)
            plt.clf()
            if i in betainds:
                plt.plot(range(1, totalsamples + 1), np.absolute(avgsamplebetas), 'bo', markersize=5, label="Random SNP Samples")
                plt.plot((0, totalsamples + 1), (np.absolute(avgbeta), np.absolute(avgbeta)), 'r-', label=disease1 + " SNPs")
            else:
                plt.plot(range(1, totalsamples + 1), np.log10(np.absolute(avgsamplebetas)), 'bo', markersize=5, label="Random SNP Samples")
                plt.plot((0, totalsamples + 1), (np.log10(np.absolute(avgbeta)), np.log10(np.absolute(avgbeta))), 'r-', label=disease1 + " SNPs")
            plt.title("Average Effect Size of "+ disease1 + " SNPs vs " + str(totalsamples) + " Random Samples", fontsize=10)
            plt.xlabel("Sample")
            if i in betainds:
                plt.ylabel("|Average Effect Size| (Beta)")
            else:
                plt.ylabel("log|Average Effect Size| (Odds Ratio)")
            plt.xlim(0, totalsamples + 1)
            plt.legend(loc="upper right", handlelength=1, handleheight=1, fontsize=5)
            plt.savefig("plots/scatterplots/" + disease1 + "scatter.pdf")

        # Get p-value of disease1 SNPs being linked to disease2

        # Get p-val for normal dist assumption
        samplemean = np.mean(avgsamplebetas)
        sampledev = np.std(avgsamplebetas)
        zscore = (avgbeta - samplemean) / sampledev
        npval = norm.sf(zscore)

        # P-val for permutation test using the random samples
        numgreaterthan = len([beta for beta in avgsamplebetas if avgbeta > beta])
        pval = float(totalsamples + 1 - numgreaterthan) / (totalsamples + 1)

        # Store cross disease analysis p-values and other information
        crossdiseases[0][i][j] = numsnpbetas
        crossdiseases[1][i][j] = avgbeta
        crossdiseases[2][i][j] = pval
        crossdiseases[3][i][j] = -np.log10(pval)
        crossdiseases[4][i][j] = npval

Furthermore, the Kolmogorov-Smirnov p-values between the disease 1 linked SNPs and the whole distribution of all SNPs with effect sizes for disease 1 are computed which basically tells how likely it is that the two samples come from the same distribution. Lists of all SNPs that have a known effect size for each disease are also found and stored for later plots.

In [8]:
# Lists of all SNPs for each disease and their averages
alldiseasesnpbetas = []
alldiseasesnpsavg = []

ksstats = [] # list of KS pvals of each disease's linked SNPs with whole distribution of SNPs with effect size for that disease

# Only want to plot values that are within two standard deviations to have clear ranges on plots and not saturate axes
for i in range(numdiseases):
    diseasebetas = allsnpbetas[diseases[i]].values()
    diseaseavg = np.mean(diseasebetas)
    diseasestd = np.std(diseasebetas)
    ndiseasebetas = [beta for beta in diseasebetas if beta <= diseaseavg + (2 * diseasestd) and beta >= diseaseavg - (2 * diseasestd)]
    ndiseaseavg = np.mean(ndiseasebetas)
    alldiseasesnpbetas.append(ndiseasebetas)
    alldiseasesnpsavg.append(ndiseaseavg)
    ksstat, kspval = ks_2samp(diseasesnpbetas[i], diseasebetas)
    ksstats.append(kspval)

All of this information regarding number of each disesase linked SNPs that have effect size for all other diseases, cross disease p-values, KS p-value, and which diseases each set of disease-linked SNPs are significant in are all printed to an output file.

In [9]:
# Write results to file
results = open("crossdiseases.txt", "w")

results.write("Number of Row Disease SNPs with Known Effect Size for Column Disease SNPs, Average Effect Size of Row Disease SNPs for Column Disease, P-Value for Row Disease Snps Effect Size for Column Disease Matrices\n\n")

for i in range(numdiseases):
    results.write("Number of " + diseases[i] + " SNPs: " + str(numsnps[i]) + "\n")
results.write("\n")

for disease in diseases:
    results.write(disease + "\t")
results.write("\n\n")

numsnpbetasmatrix = crossdiseases[0]
results.write("Number of Row Disease SNPs with Known Effect Size for Column Disease\n")
for i in range(numdiseases):
    for j in range(numdiseases):
        results.write(str(numsnpbetasmatrix[i][j]) + "\t")
    results.write("\n")
results.write("\n")

avgbetamatrix = crossdiseases[1]
results.write("Average Effect Size of Row Disease SNPs for Column Disease\n")
for i in range(numdiseases):
    for j in range(numdiseases):
        results.write(str(avgbetamatrix[i][j]) + "\t")
    results.write("\n")
results.write("\n")

npvalmatrix = crossdiseases[4]
results.write("P-Value of Row Disease SNPs for Column Disease Using Normal Approximation\n")
for i in range(numdiseases):
    for j in range(numdiseases):
        results.write(str(npvalmatrix[i][j]) + "\t")
    results.write("\n")
results.write("\n")

pvalmatrix = crossdiseases[2]
results.write("P-Value of Row Disease SNPs for Column Disease Using Permutation Test\n")
for i in range(numdiseases):
    for j in range(numdiseases):
        results.write(str(pvalmatrix[i][j]) + "\t")
    results.write("\n")
results.write("\n")

logmatrix = crossdiseases[3]
results.write("-log of P-Value of Row Disease SNPs for Column Disease Using Permutation Test\n")
for i in range(numdiseases):
    for j in range(numdiseases):
        results.write(str(logmatrix[i][j]) + "\t")
    results.write("\n")
results.write("\n")

results.write("KS P-Value of Disease SNPs with All SNPs with Effect Size for that Disease\n")
for i in range(numdiseases):
    results.write(diseases[i] + ": " + str(ksstats[i]) + "\n")
results.write("\n")

significancethresh = 0.05

for i in range(numdiseases):
    results.write("Other Diseases where " + diseases[i] + " SNPs have Significant Normal Approximation P-Values: ")
    for j in range(numdiseases):
        if npvalmatrix[i][j] <= significancethresh:
            results.write(diseases[j] + ": " + str(npvalmatrix[i][j]) + ", ")
    results.write("\n")
results.write("\n")

for i in range(numdiseases):
    results.write("Other Diseases where " + diseases[i] + " SNPs have Significant Permutation Test P-Values: ")
    for j in range(numdiseases):
        if pvalmatrix[i][j] <= significancethresh:
            results.write(diseases[j] + ": " + str(pvalmatrix[i][j]) + ", ")
    results.write("\n")
results.write("\n")

results.close()

SNPs linked to each disease are also printed to an output file.

In [10]:
diseasesnpfile = open("diseasesnps.txt", "w")

diseasesnpfile.write("SNP\tPheno\tPaper\tP-Value\n")
for i in range(numdiseases):
    diseasesnpfile.write(diseases[i] + " related SNPs:\n")
    for snp in diseasesnps[i]:
        snpinfo = snpsinfo[snp]
        diseasesnpfile.write(snpinfo[0] + "\t" + snpinfo[1] + "\t" + snpinfo[2] + "\t" + snpinfo[3] + "\n")
    diseasesnpfile.write("\n")

diseasesnpfile.close()

Create a heatmap of cross disease p-values.

In [11]:
plt.clf()
plt.imshow(logmatrix, interpolation='nearest')
plt.colorbar()
plt.savefig("plots/heatmap.pdf")

# Creating Plots

Box and Violin plots are created to compare distributions of disease-linked SNPs with the distributions of all SNPs with known effect sizes for that disease. For these plots, we want to plot the absolute value abs(beta) if the effect size is beta and the absolute value of the log abs(log(odds)) if the effect size is odd ratio for clearer comparisons of distributions.

Lists of data are created for plots with all diseases on them.

In [12]:
# Lists of lists of effect sizes for disease linked SNPs and all SNPs with effect size based on whether beta or OR is used
betadata = []
oddsdata = []

# Lists of indices of disease data for accurate labeling on plots
betapos = []
oddspos = []

pos = range(1, 2 * numdiseases + 1)
labels = []

betalabels = []
oddslabels = []

# Add data to lists depending on beta or odds ratio
for i in range(numdiseases):
    if i in betainds:
        betadata.append(np.absolute(diseasesnpbetas[i]))
        betadata.append(np.absolute(alldiseasesnpbetas[i]))
        betapos.append(i * 2 + 1)
        betapos.append(i * 2 + 2)
        betalabels.append(diseases[i][:3])
        betalabels.append("All " + diseases[i][:3])
    else:
        oddsdata.append(np.absolute(np.log10(diseasesnpbetas[i])))
        oddsdata.append(np.absolute(np.log10(alldiseasesnpbetas[i])))
        oddspos.append(i * 2 + 1)
        oddspos.append(i * 2 + 2)
        oddslabels.append(diseases[i][:3])
        oddslabels.append("All " + diseases[i][:3])
    labels.append(diseases[i][:3])
    labels.append("All " + diseases[i][:3])

All the violin plots for each disease are plotted onto one figure.

In [13]:
# All violin plots on one plot using two different y-axes, one for beta and one for odds ratio
plt.clf()
fig, ax1 = plt.subplots()
a1 = ax1.violinplot(betadata, betapos, points=40, widths=.5, showmeans=True, showextrema=True)
ax1.set_ylabel('|Beta|', color='r')
ax1.set_xlabel("Disease Distribution of SNPs")
values = [val for sublist in betadata for val in sublist]
ax1.set_ylim([np.min(values) - .1, np.max(values) + .1])
ax1.tick_params(axis='x', labelsize=5)

a1["cbars"].set_color('r')
a1["cmeans"].set_color('r')
a1["cmins"].set_color('r')
a1["cmaxes"].set_color('r')

ax2 = ax1.twinx()
a2 = ax2.violinplot(oddsdata, oddspos, points=40, widths=.4, showmeans=True, showextrema=True)
ax2.set_ylabel('|log(Odds Ratio)|', color='b')
values = [val for sublist in oddsdata for val in sublist]
ax2.set_ylim([np.min(values) - .1, np.max(values) + .1])

a2["cbars"].set_color('b')
a2["cmeans"].set_color('b')
a2["cmins"].set_color('b')
a2["cmaxes"].set_color('b')

plt.xticks(pos, labels)
plt.title("All Disease Violin Plots", fontsize=10)
fig.tight_layout()
plt.savefig("plots/allviolin.pdf")

Two separate violin plots are created, one containing plots diseases with beta as effect size, the other containing plots diseases with odds ratio as effect size.

In [14]:
# Two Separate Violin Plots
newbetapos = range(1, len(betadata) + 1)
newoddspos = range(1, len(oddsdata) + 1)

plt.clf()
a1 = plt.violinplot(betadata, newbetapos, points=40, widths=.5, showmeans=True, showextrema=True)
plt.ylabel('|Beta|', color='r')
plt.xlabel("Disease Distribution of SNPs")
values = [val for sublist in betadata for val in sublist]
plt.ylim([np.min(values) - .1, np.max(values) + .1])
plt.tick_params(axis='x', labelsize=5)

a1["cbars"].set_color('r')
a1["cmeans"].set_color('r')
a1["cmins"].set_color('r')
a1["cmaxes"].set_color('r')

plt.xticks(newbetapos, betalabels)
plt.title("All Disease Violin Plots with Beta Effect Size", fontsize=10)
plt.savefig("plots/betaviolin.pdf")


plt.clf()
a2 = plt.violinplot(oddsdata, newoddspos, points=40, widths=.5, showmeans=True, showextrema=True)
plt.ylabel('|log(Odds Ratio)|', color='b')
values = [val for sublist in oddsdata for val in sublist]
plt.ylim([np.min(values) - .1, np.max(values) + .1])

a2["cbars"].set_color('b')
a2["cmeans"].set_color('b')
a2["cmins"].set_color('b')
a2["cmaxes"].set_color('b')

plt.xticks(newoddspos, oddslabels)
plt.title("All Disease Violin Plots", fontsize=10)
plt.savefig("plots/oddsviolin.pdf")

All scatterplots are also plotted onto one figure.

In [15]:
cols = 3
rows = int(np.ceil(float(numdiseases) / cols))

plt.clf()
fig, axes = plt.subplots(nrows=rows, ncols=cols)
diseaseind = 0
for i in range(rows):
    for j in range(cols):
        if diseaseind >= numdiseases:
            diseaseind += 1
            continue
        if diseaseind in betainds:
            axes[i, j].plot(range(1, totalsamples + 1), np.absolute(allavgsamplebetas[diseaseind]), 'bo', markersize=2, label="Random SNP Samples")
            axes[i, j].plot((0, totalsamples + 1), (np.absolute(allavgbetas[diseaseind]), np.absolute(allavgbetas[diseaseind])), 'r-', label="Disease SNPs")
        else:
            axes[i, j].plot(range(1, totalsamples + 1), np.absolute(np.log10(allavgsamplebetas[diseaseind])), 'bo', markersize=2, label="Random SNP Samples")
            axes[i, j].plot((0, totalsamples + 1), (np.absolute(np.log10(allavgbetas[diseaseind])), np.absolute(np.log10(allavgbetas[diseaseind]))), 'r-', label="Disease SNPs")
        axes[i, j].set_title(diseases[diseaseind], fontsize=5)
        axes[i, j].set_xlim(0, totalsamples + 1)
        axes[i, j].tick_params(axis='both', labelsize=3)
        if diseaseind in betainds:
            axes[i, j].set_ylabel("|Avg Effect Size| (Beta)", fontsize=5)
        else:
            axes[i, j].set_ylabel("|log(Avg Effect Size)| (OR)", fontsize=5)
        diseaseind +=1

fig.suptitle("Disease Scatter Plots")
plt.savefig("plots/allscatter.pdf")

Individual violin and box plots are also created for each disease.

In [16]:
# Individual plots
pos = [1, 2]
labels = ["Disease-Linked SNPs", "All SNPs"]

for i in range(numdiseases):
    # Violin plots
    plt.clf()
    if i in betainds:
        plt.violinplot([np.absolute(diseasesnpbetas[i]), np.absolute(alldiseasesnpbetas[i])], pos, points=40, widths=.5, showmeans=True, showextrema=True)
        plt.ylabel("|Effect Size| (Beta)")
    else:
        plt.violinplot([np.absolute(np.log10(diseasesnpbetas[i])), np.absolute(np.log10(alldiseasesnpbetas[i]))], pos, points=40, widths=.5, showmeans=True, showextrema=True)
        plt.ylabel("|log(Effect Size)| (OR)")
    plt.title(diseases[i] + " Violin Plot", fontsize=10)
    plt.xticks(pos, labels)
    plt.xlabel("Distribution")
    plt.savefig("plots/violinplots/" + diseases[i] + "violin.pdf")

    # Box pots
    plt.clf()
    if i in betainds:
        plt.boxplot([np.absolute(diseasesnpbetas[i]), np.absolute(alldiseasesnpbetas[i])], 0, '')
        plt.ylabel("|Effect Size| (Beta)")
    else:
        plt.boxplot([np.absolute(np.log10(diseasesnpbetas[i])), np.absolute(np.log10(alldiseasesnpbetas[i]))], 0, '')
        plt.ylabel("log|Effect Size| (OR)")
    plt.xticks(pos, labels)
    plt.xlabel("Distribution")

    plt.title(diseases[i] + " Box Plot", fontsize=10)
    plt.savefig("plots/boxplots/" + diseases[i] + "box.pdf")